强化学习是一类算法, 是让计算机实现从一开始什么都不懂, 脑袋里没有一点想法, 通过不断地尝试, 从错误中学习, 最后找到规律, 学会了达到目的的方法. 这就是一个完整的强化学习过程. 

Q-learning

DQN

我们可以将状态和动作当成神经网络的输入, 然后经过神经网络分析后得到动作的 Q 值, 这样我们就没必要在表格中记录 Q 值, 而是直接使用神经网络生成 Q 值. 还有一种形式的是这样, 我们也能只输入状态值, 输出所有的动作值, 然后按照 Q learning 的原则, 直接选择拥有最大值的动作当做下一步要做的动作. 我们可以想象, 神经网络接受外部的信息, 相当于眼睛鼻子耳朵收集信息, 然后通过大脑加工输出每种动作的值, 最后通过强化学习的方式选择动作.



DDPG是google DeepMind团队提出的一种用于输出确定性动作的算法，它解决了Actor-Critic 神经网络每次参数更新前后都存在相关性，导致神经网络只能片面的看待问题这一缺点。同时也解决了DQN不能用于连续性动作的缺点。

## Deep Deterministic Policy Gradient

### 简介

DDPG即深度确定性策略梯度算法。也是一种可以解决连续性控制问题的方法。属于model-free、off-policy、policy-based的方法。

DDPG我们可以拆开来看，Deep，是说明需要**神经网络**。Deterministic的意思就是最终确定地**只输出一个动作**。Policy Gradient我们已经知道是**策略梯度算法**。DDPG可以看成是DQN的扩展版，不同的是，以往的DQN在最终输出的是一个动作向量，对于DDPG是最终确定地只输出一个动作。而且，DDPG让 DQN 可以扩展到连续的动作空间。

### 网络结构

DDPG的结构形式类似Actor-Critic。DDPG可以分为**策略网络**和**价值网络**两个大网络。DDPG延续DQN了固定目标网络的思想，每个网络再细分为目标网络和现实网络。不过目标网络的更新上有些不同。下面详细分析。

先来看策略网络，也就是Actor。Actor输出的是一个确定性的动作，产生这个确定性动作的网络定义为 @todo 。以往的policy gradient采取的是随机策略，每一次获取动作都需要对当前的最优策略的分布进行采样，而DDPG采取的则是确定性策略，直接通过函数 @todo 确定。Actor的估计网络就是 @todo ，@todo 就是神经网络的参数，这个估计网络就是用来输出实时的动作。此外，Actor还有一个相同结构但不同参数的目标网络，是用来更新价值网络Critic的。两个网络都是输出动作action。

再来看价值网络，也就是Critic。它的作用就是来拟合价值函数 @todo 。同样也有一个估计网络和一个目标网络。这两个网络在输出端都输出当前状态的价值 q-value，在输入端则有所不同。Critic的目标网络输入有两个参数，分别是当前状态的观测值和Actor的目标网络输出的动作action。Critic的估计网络的输入则是当前Actor的估计网络输出的动作action。目标网络是用来计算 @todo ,

下面我们通过一张图来直观的理解下整个过程：


我们可以看到，价值网络的更新是基于TD-error的梯度下降，Critic作为评委，一开始也不知道Actor输出的动作是否足够好，它也需要一步一步学习给出准确的打分，所以借助于目标网络拟合的下一时刻的价值 @todo ，以及真实的收益r,我们可以得到 @todo ，让 @todo 减去当前 @todo 求均方差，则可以构造出Loss。更新方法其实和DQN差不多的，唯一不同的地方在于目标网络的参数在DDPG算法中是缓慢更新的，而不是在DQN中每隔N步将现有网络的参数直接复制过来。

而策略网络（Actor）的更新是基于梯度上升，因为Actor的目标就是找到一个动作a能使得输出的价值Q最大，所以优化策略网络的梯度就是要最大化价值网络输出的这个Q值。Loss函数就加上一个负号方便最小化误差。

值得注意的是，DDQG也借鉴了DQN的经验回放 (Experience Replay)的技巧。DDPG同样会把一段时间的序列 @todo 存储到经验池。每次训练的时候，从经验池随机采样一个minibatch来训练就好。

### 算法流程

伪代码如下：


首先，初始化Actor和Critic以及其各自的目标网络共4个网络以及经验池replay buffer R。

在Actor网络输出动作时，DDPG通过添加随机噪声的方式实现exploration，可以让智能体更好的探索潜在的最优策略。之后是采取经验回放的技巧。把智能体与环境交互的数据 
 存储到R。随后每次训练从中随机采样一个minibatch。

在参数更新上，先利用Critic的目标网络 
 来计算目标值 
 ，利用 
 与当前Q值的均方误差构造损失函数，进行梯度更新。对于Actor的策略网络，其实就是把Actor的确定性动作函数代进Q-function的a，然后求梯度，最后是更新目标网络。

### 小结

简单来说：DQN + Actor-Critic => Deep Deterministic Policy Gradient (DDPG)。实际上DDPG其实更接近DQN，只是采用了类似Actor-Critic的结构。DDPG吸收了Actor-Critic中策略梯度单步更新的优点，同时还吸收了DQN对Q值估计的技巧。DDPG 最大的优势就是能够在**连续动作**上更有效地学习。

参考文献：

Silver, David, et al. "Deterministic policy gradient algorithms." ICML. 2014.（前序工作）

Lillicrap, Timothy P., et al. "Continuous control with deep reinforcement learning." arXiv preprint arXiv:1509.02971 (2015)

In [7]:
import tensorflow
tf = tensorflow.compat.v1
tf.disable_eager_execution()

In [2]:
import gym
import time
import numpy as np

In [3]:
MAX_EPISODES = 200
MAX_EP_STEPS = 200
LR_A = 0.001    # learning rate for actor
LR_C = 0.002    # learning rate for critic
GAMMA = 0.9     # reward discount
TAU = 0.01      # soft replacement
MEMORY_CAPACITY = 10000
BATCH_SIZE = 32

RENDER = False
ENV_NAME = 'Pendulum-v0'

In [ ]:
class PrimalDualDDPG(object):
    def __init__(self):
        self.step = 0

In [4]:
class DDPG(object):
    def __init__(self, action_dim, state_dim, action_bound,):
        self.memory = np.zeros((MEMORY_CAPACITY, state_dim * 2 + action_dim + 1), dtype=np.float32)
        self.pointer = 0
        self.sess = tf.Session()

        self.action_dim, self.state_dim, self.action_bound = action_dim, state_dim, action_bound,
        self.S = tf.placeholder(tf.float32, [None, state_dim], 's')
        self.S_ = tf.placeholder(tf.float32, [None, state_dim], 's_')
        self.R = tf.placeholder(tf.float32, [None, 1], 'r')

        with tf.variable_scope('Actor'):
            self.a = self._build_a(self.S, scope='eval', trainable=True)
            a_ = self._build_a(self.S_, scope='target', trainable=False)
        with tf.variable_scope('Critic'):
            # assign self.a = a in memory when calculating q for td_error,
            # otherwise the self.a is from Actor when updating Actor
            q = self._build_c(self.S, self.a, scope='eval', trainable=True)
            q_ = self._build_c(self.S_, a_, scope='target', trainable=False)

        # networks parameters
        self.ae_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Actor/eval')
        self.at_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Actor/target')
        self.ce_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Critic/eval')
        self.ct_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Critic/target')

        # target net replacement
        self.soft_replace = [tf.assign(t, (1 - TAU) * t + TAU * e)
                             for t, e in zip(self.at_params + self.ct_params, self.ae_params + self.ce_params)]

        q_target = self.R + GAMMA * q_
        # in the feed_dic for the td_error, the self.a should change to actions in memory
        td_error = tf.losses.mean_squared_error(labels=q_target, predictions=q)
        self.ctrain = tf.train.AdamOptimizer(LR_C).minimize(td_error, var_list=self.ce_params)

        a_loss = - tf.reduce_mean(q)    # maximize the q
        self.atrain = tf.train.AdamOptimizer(LR_A).minimize(a_loss, var_list=self.ae_params)

        self.sess.run(tf.global_variables_initializer())

    def choose_action(self, s):
        return self.sess.run(self.a, {self.S: s[np.newaxis, :]})[0]

    def learn(self):
        # soft target replacement
        self.sess.run(self.soft_replace)

        indices = np.random.choice(MEMORY_CAPACITY, size=BATCH_SIZE)
        bt = self.memory[indices, :]
        bs = bt[:, :self.state_dim]
        ba = bt[:, self.state_dim: self.state_dim + self.action_dim]
        br = bt[:, -self.state_dim - 1: -self.state_dim]
        bs_ = bt[:, -self.state_dim:]

        self.sess.run(self.atrain, {self.S: bs})
        self.sess.run(self.ctrain, {self.S: bs, self.a: ba, self.R: br, self.S_: bs_})

    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, a, [r], s_))
        index = self.pointer % MEMORY_CAPACITY  # replace the old memory with new memory
        self.memory[index, :] = transition
        self.pointer += 1

    def _build_a(self, s, scope, trainable):
        with tf.variable_scope(scope):
            net = tf.layers.dense(s, 30, activation=tf.nn.relu, name='l1', trainable=trainable)
            a = tf.layers.dense(net, self.action_dim, activation=tf.nn.tanh, name='a', trainable=trainable)
            return tf.multiply(a, self.action_bound, name='scaled_a')

    def _build_c(self, s, a, scope, trainable):
        with tf.variable_scope(scope):
            n_l1 = 30
            w1_s = tf.get_variable('w1_s', [self.state_dim, n_l1], trainable=trainable)
            w1_a = tf.get_variable('w1_a', [self.action_dim, n_l1], trainable=trainable)
            b1 = tf.get_variable('b1', [1, n_l1], trainable=trainable)
            net = tf.nn.relu(tf.matmul(s, w1_s) + tf.matmul(a, w1_a) + b1)
            return tf.layers.dense(net, 1, trainable=trainable)  # Q(s,a)


In [8]:
env = gym.make(ENV_NAME)
env = env.unwrapped
env.seed(1)

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
action_bound = env.action_space.high

ddpg = DDPG(action_dim, state_dim, action_bound)

var = 3  # control exploration
t1 = time.time()
for i in range(MAX_EPISODES):
    s = env.reset()
    ep_reward = 0
    for j in range(MAX_EP_STEPS):
        if RENDER:
            env.render()

        # Add exploration noise
        a = ddpg.choose_action(s)
        a = np.clip(np.random.normal(a, var), -2, 2)    # add randomness to action selection for exploration
        s_, r, done, info = env.step(a)

        ddpg.store_transition(s, a, r / 10, s_)

        if ddpg.pointer > MEMORY_CAPACITY:
            var *= .9995    # decay the action randomness
            ddpg.learn()

        s = s_
        ep_reward += r
        if j == MAX_EP_STEPS-1:
            print('Episode:', i, ' Reward: %i' % int(ep_reward), 'Explore: %.2f' % var, )
            # if ep_reward > -300:RENDER = True
            break
print('Running time: ', time.time() - t1)

[2023-01-25 09:59:35,241] Making new env: Pendulum-v0
d:\Python\Python310\lib\site-packages\gym\envs\registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_6696\2257476852.py:66: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  net = tf.layers.dense(s, 30, activation=tf.nn.relu, name='l1', trainable=trainable)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_6696\2257476852.py:67: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  a = tf.layers.dense(net, self.a_dim, activation=tf.nn.tanh, name='a', trainable=trainable)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_6696\2257476852.py:77: UserWarning: `tf.layers.dense` is deprecated and will be removed in a fut

Episode: 0  Reward: -1214 Explore: 3.00
Episode: 1  Reward: -1308 Explore: 3.00
Episode: 2  Reward: -1316 Explore: 3.00
Episode: 3  Reward: -1438 Explore: 3.00
Episode: 4  Reward: -1304 Explore: 3.00
Episode: 5  Reward: -1288 Explore: 3.00
Episode: 6  Reward: -1277 Explore: 3.00
Episode: 7  Reward: -1270 Explore: 3.00
Episode: 8  Reward: -1276 Explore: 3.00
Episode: 9  Reward: -1265 Explore: 3.00
Episode: 10  Reward: -1292 Explore: 3.00
Episode: 11  Reward: -1179 Explore: 3.00
Episode: 12  Reward: -1509 Explore: 3.00
Episode: 13  Reward: -1324 Explore: 3.00
Episode: 14  Reward: -1168 Explore: 3.00
Episode: 15  Reward: -1400 Explore: 3.00
Episode: 16  Reward: -1282 Explore: 3.00
Episode: 17  Reward: -1292 Explore: 3.00
Episode: 18  Reward: -1302 Explore: 3.00
Episode: 19  Reward: -1282 Explore: 3.00
Episode: 20  Reward: -1330 Explore: 3.00
Episode: 21  Reward: -1314 Explore: 3.00
Episode: 22  Reward: -1212 Explore: 3.00
Episode: 23  Reward: -1296 Explore: 3.00
Episode: 24  Reward: -1225